<a href="https://colab.research.google.com/github/IvoPascoal/IvoP_20160462/blob/master/Regressao_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import numpy as np 
import pandas as pd
from pandas import Series
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from math import sqrt
from sklearn.preprocessing import MinMaxScaler



url = 'https://raw.githubusercontent.com/IvoPascoal/IvoP_20160462/master/DUQ_hourly.csv'
df1 = pd.read_csv(url, index_col=[0], parse_dates=[0])

'''df1.plot(style='.', figsize=(20,10), color= 'b', title='Esparsidade')'''



df1 = df1.groupby(pd.Grouper(freq="D")).sum()


def create_features(df1, label=None):
  
  df1 = df1.copy()
  df1['Data'] = df1.index
  df1['Dia da Semana'] = df1['Data'].dt.dayofweek
  df1['Trimestre'] = df1['Data'].dt.quarter
  df1['Mês'] = df1['Data'].dt.month
  df1['Ano'] = df1['Data'].dt.year
  df1['Dia do Ano'] = df1['Data'].dt.dayofyear
  df1['Dia do Mês'] = df1['Data'].dt.day

    
  X = df1[['Dia da Semana','Trimestre','Mês','Ano','Dia do Ano','Dia do Mês']]
    
  if label:
    y = df1[label]
    return X, y
  return X

X, y = create_features(df1, label='DUQ_MW')

features_and_target = pd.concat([X, y], axis=1)

features_and_target.head(100000)

treino = features_and_target.loc[:'2017-12-31']
teste = features_and_target.loc['2018-01-01':]

X_treino = pd.DataFrame(treino.iloc[:,:-1])
y_treino = pd.DataFrame(treino['DUQ_MW'])

X_teste = pd.DataFrame(teste.iloc[:,:-1])
y_teste = pd.DataFrame(teste['DUQ_MW'])

scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(y_treino)

normalized_y_treino = scaler.transform(y_treino)

normalized_y_teste = scaler.transform(y_teste)

regr = linear_model.LinearRegression()

regr.fit(X_treino, normalized_y_treino)

y_pred = regr.predict(X_teste)

inversed_y_pred = scaler.inverse_transform(y_pred)
inversed_y_pred.shape #array 215,1
                                                                                                                           
mean_squared_error(y_true=teste['DUQ_MW'], y_pred=inversed_y_pred)

mean_absolute_error(y_true=teste['DUQ_MW'], y_pred=inversed_y_pred)

def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(y_true=y_teste, y_pred=inversed_y_pred)

r2_score(y_teste.values, inversed_y_pred)

verdadeiro_previsto = pd.DataFrame({'Actual': y_teste, 'Predicted': inversed_y_pred})  

plt.scatter(X_teste, y_teste,  color='black')
plt.plot(y_teste, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()






0.013568990129789849